<a href="https://colab.research.google.com/github/mohanapriyaab/datascience/blob/main/webscraping_using_beautiful_soup_and_selenium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install beautifulsoup4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 22.8 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,604 kB]
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubu

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time


In [ ]:

options = Options()
options.add_argument("--headless")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--no-sandbox")


driver = webdriver.Chrome(options=options)


In [ ]:

channel_urls = [
    'https://www.youtube.com/@BlackSheepTamil',
    'https://www.youtube.com/@VillageCookingChannel',
    'https://www.youtube.com/@Micset',
    'https://www.youtube.com/@BehindwoodsTV',
    'https://www.youtube.com/@MadrasCentral',
    'https://www.youtube.com/@TamilVoiceStudio',
    'https://www.youtube.com/@Nakkalites',
    'https://www.youtube.com/@ErumaSaani'
]



In [ ]:

data = []

for url in channel_urls:
    try:
        driver.get(url)
        time.sleep(5)
        soup = BeautifulSoup(driver.page_source, 'html.parser')


        title = soup.find("meta", property="og:title")
        name = title["content"] if title else "N/A"


        try:
            subs = driver.find_element(By.XPATH, '//*[@id="subscriber-count"]').text
        except:
            subs = "N/A"


        driver.get(url + "/about")
        time.sleep(5)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        stats = soup.find_all("yt-formatted-string", {"id": "text", "class": "style-scope ytd-channel-about-metadata-renderer"})

        video_count = "N/A"
        for s in stats:
            if "videos" in s.text:
                video_count = s.text
                break

        driver.get(url + "/videos")
        time.sleep(5)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        first_video = soup.find('a', id='video-title')
        video_url = "https://www.youtube.com" + first_video['href'] if first_video else None

        likes = "N/A"
        if video_url:
            driver.get(video_url)
            time.sleep(5)
            try:
                like_btn = driver.find_element(By.XPATH, '(//yt-formatted-string[@id="text"][@class="style-scope ytd-toggle-button-renderer style-text"])[1]')
                likes = like_btn.text
            except:
                likes = "N/A"

        data.append({
            "Channel": name,
            "Subscribers": subs,
            "Videos": video_count,
            "Latest Video Likes": likes
        })

        print(f"✔️ Scraped: {name}")

    except Exception as e:
        print(f"❌ Error scraping {url}: {e}")

driver.quit()


✔️ Scraped: Black Sheep
✔️ Scraped: Village Cooking Channel
✔️ Scraped: Arul Kumar
✔️ Scraped: Behindwoods TV
✔️ Scraped: Madras Central
✔️ Scraped: N/A
✔️ Scraped: Nakkalites
✔️ Scraped: Eruma Saani


In [ ]:

import pandas as pd

df = pd.DataFrame(data)
csv_path = "/content/tamil_youtube_channel_data.csv"
df.to_csv(csv_path, index=False)

print(f"\n📁 CSV saved to: {csv_path}")



📁 CSV saved to: /content/tamil_youtube_channel_data.csv


In [ ]:
from google.colab import files
files.download("/content/tamil_youtube_channel_data.csv")


FileNotFoundError: Cannot find file: /content/tamil_youtube_channel_data.csv